### Backtesting on Synthetic Data

### Loading Libraries

In [1]:
# Randomness
import random

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
from pandas import Timestamp

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
%matplotlib inline

# Date & Time
from datetime import datetime, timedelta

# Typing
from typing import Tuple, List, Dict, Union, Optional, Any, Generator

# Scikit-Learn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve

# Scientific Statistical Python
from scipy.stats import jarque_bera
from scipy.stats import rv_continuous, kstest

### Numerical Determination of Optimal Trading Rules
#### Setting Batch & Implementation

In [4]:
def batch(
    coeffs: Dict[str, float], nIter: int = 1e4, maxHP: int = 100, rPT: np.ndarray = np.linspace(0.5, 10, 20),
    rSLm: np.ndarray = np.linspace(0.5, 10, 20), seed: float = 0.0) -> list:

    phi, output1 = 2 ** (-1.0 / coeffs['hl']), []
    for comb_ in product(rPT, rSLm):
        output2 = []
        for iter_ in range(int(nIter)):
            p, hp, count = seed, 0, 0
            while True:
                p = (1 - phi) * coeffs['forecast'] + phi * p + coeffs['sigma'] * gauss(0, 1)
                cP = p - seed
                hp += 1
                if cP > comb_[0] or cP < -comb_[1] or hp > maxHP:
                    output2.append(cP)
                    break
        mean, std = np.mean(output2), np.std(output2)
        output1.append((comb_[0], comb_[1], mean, std, mean / std))
    return output1

#### Experimental Results

In [1]:
def iterate_parameters():
    rPT = rSLm = np.linspace(0,10,21)
    count = 0
    for prod_ in product([10, 5, 0, -5, -10], [5, 10, 25, 50, 100]):
        count += 1
        coeffs = {'forecast': prod_[0], 'hl': prod_[1], 'sigma': 1}
        output = batch(coeffs, nIter=1e4, maxHP=100, rPT=rPT, rSLm=rSLm)
    return output